In [ ]:
class WandBMonitor():
    """
    A W&B class to save images to WandB.
    """
    
    def __init__(self, n_images=1):
        self.n_images = n_images
        
    def get_wandb_image(self, images):
        images = (images * 127.5 + 127.5)
        images = wandb.Image(images)
        return images
        
    
    def on_epoch_end(self, epoch, logs):
        # Real to Monet Generated Images
        for i, image in enumerate(photo_ds_eval.take(self.n_images)):
            monet_pred = monet_generator.predict(image)
            monet_wandb = self.get_wandb_image(monet_pred)
        
        # Monet to Real Generated Images
        for i, image in enumerate(monet_ds_eval.take(self.n_images)):
            real_pred = photo_generator.predict(image)
            real_wandb = self.get_wandb_image(real_pred)
            
        # Log losses
        wandb.log({
            "real_to_monet": monet_wandb,
            "monet_to_real": real_wandb,
            "monet_gen_loss": tf.reduce_mean(logs["monet_gen_loss"]),
            "photo_gen_loss": tf.reduce_mean(logs["photo_gen_loss"]),
            "monet_disc_loss": tf.reduce_mean(logs["monet_disc_loss"]),
            "photo_disc_loss": tf.reduce_mean(logs["photo_disc_loss"]),
        }, step=epoch)